# Plotting Wind Properties
As described under [Models](../output/model.rst), Python saves wind properties in binary wind_save files. This notebook explains how to read and plot wind variables for the ```cv_standard``` file found in the examples. Before running the python commands, you need to run the model from the command line. I suggest running the following commands, after you have compiled python:

    mkdir cv_test
    cd cv_test
    cp $PYTHON/examples/basic/cv_standard.pf .
    py cv_standard </code>

The model will take about 5 minutes to run on a single core. It will not converge, but will give us a model to use as an example. You should then run ```windsave2table``` on the output

    windsave2table cv_standard

which will create a series of ascii files containing key variables in the wind cells. We will use these ascii files for our plots. 

## Make A Basic Quick Look Wind Plot

The simplest way to make a quick look plot of the electron temperature is using the ```plot_wind.py``` routine in ```$PYTHON/py_progs```. In this example, I will assume py_progs has been added to ```$PATH``` and to ```$PYTHONPATH```. ```plot_wind.py``` can be run from the command line using

    plot_wind.py cv_standard t_e
    
where the second argument is the variable to plot. Alternatively, it can be run from within a python script by doing (where we are now assuming you are running this code from one directory above cv_test):

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt 
import numpy as np 
import plot_wind
fname = "cv_test/cv_standard.master.txt"
plot_wind.doit(fname, var="t_e")

## More detailed/customisable plots

You may, however, wish to get more direct access to the data, which can be done easily by reading in the ```cv_standard.master.txt``` file, for example using ```astropy```. In the next code block, we read in the data file and print out the columns.

In [6]:
import matplotlib.pyplot as plt
import astropy.io.ascii as io 

data = io.read(fname)

print (data.colnames)

 The ```py_plot_util``` script in py_progs comes with a handy guide to the main columns in the .master.txt file and returns a dictionary containing the description for all variables.

In [7]:
import py_plot_util as util  
descr = util.get_windsave_descriptions(data)

```py_plot_util``` also contains some routines for reshaping and masking arrays and so on. One of the most useful for plotting is the ```wind_to_masked``` function which turns the raw 1D flattened data into a masked 2D array with the right shape which can be easily used with ```pcolormesh``` and so on. Here's an example plot of the electron density in the model.

In [13]:
x, z, ne, inwind = util.wind_to_masked(data, value_string="ne", return_inwind=True)
plt.pcolormesh(x,z, np.log10(ne))
plt.loglog()
plt.xlim(1e9,1e12)
plt.ylim(1e8,1e12)
cbar = plt.colorbar()

This procedure can be used to plot any of the variables in the masterfile and is a good starting point for delving into the properties of the wind. 

To make a simple multi-panel plot of wind properties, you can use some of the routines in ```py_plot_output```. The example below plots all the variables passed in an array and saves the file as ```cv_standard_wind.png```

In [18]:
import py_plot_output as plot 
plot.make_wind_plot(data, "cv_standard_wind", var = ["ne", "t_e", "t_r", "xi", "ntot", "v_x", "h1", "c4"], shape=(4,2) )

## Plotting Ion Populations 

Ion populations outputted from ```windsave2table``` are stored in files like ```cv_standard.C.frac.txt```, where the letter before frac denotes the element. Plots of the C III ion fraction can thus be made through commands like the following, where strings like ```i05``` index the ion for each file. 


In [12]:
carbon_ion = io.read("cv_test/cv_standard.C.frac.txt")
x, z, c3_frac, inwind = util.wind_to_masked(carbon_ion, value_string="i03", return_inwind=True)
plt.pcolormesh(x,z, np.log10(c3_frac))
plt.loglog()
plt.xlim(1e9,1e12)
plt.ylim(1e8,1e12)
cbar = plt.colorbar()